In [ ]:
%run MCTS.ipynb
%run NNet_architecture.ipynb
%run Deck.ipynb
%run Board.ipynb
import time
from tensorflow.keras.models import load_model

In [ ]:
# model = create_model()
model = load_model("models/latest.h5")

### Training

In [ ]:
new_board = Board(deck)
root = MonteCarloTreeSearchNode(model, new_board, prior=0)

In [ ]:
nb_simulations = 200
nb_games = 10
max_move_per_game = np.inf

In [ ]:
node = root
games = [MonteCarloTreeSearchNode(model, Board(deck), prior=0) for i in range(nb_games)]
game = 0
i = 0
board_states = []
values = []
policies = []

while game < nb_games:
    node = games[game]
    i = 0
    while not node.board.is_game_over() and i < max_move_per_game:
        node.simulate(nb_simulations)
        
        l = [(ind, node._number_of_visits) for ind, node in node.children.items()]
        ind, action = max(l, key = lambda x:x[1])
        
        policy = np.zeros(5 * 5 * 52)
        total_visits = sum([b._number_of_visits for b in node.children.values()])
        for a, b in node.children.items():
            policy[a] = b._number_of_visits/total_visits
        
        policies.append(policy)
        board_states.append(node.board.board_state)
        values.append(node.mean_value()[0][0])
        
        # nodes.append(node)
        node = node.children[ind]
        
        plt.imshow(node.board.board_2D)
        plt.show()
        i += 1
        print("-----------", i)
    game +=1
    print("---------------------------------", game, "---------------------------------")

In [ ]:
board_states_2 = np.zeros((len(board_states), 5, 5, 10))
policies_2 = np.zeros((len(policies), 5, 5, 52))

In [ ]:
for i, board in enumerate(board_states):
    
    board_states_2[i] = board
    policies_2[i] = policies[i].reshape((5, 5, 52))

In [ ]:
board_states_2.shape

# Fit games

In [ ]:
model.fit(board_states_2, [policies_2, np.array(values)], epochs=15)

In [ ]:
model.save("models/latest.h5")

In [ ]:
import pandas as pd
pd.DataFrame(model.history.history).plot()